In [1]:
import pandas as pd

mapo_parking_df = pd.read_csv('data/mapo_parking_df.csv')
mapo_df220307 = pd.read_csv('data/mapo_df220307.csv')
mapo_df230322 = pd.read_csv('data/mapo_df230322.csv')

In [16]:
mapo_df220307.head()

,Unnamed: 0,번호,신고일,구정보,주소,유형,조치일,lat,lon
0,0,33615,2022-03-07,마포구,서울특별시 마포구 서교동 406,보도 중앙,2022-03-07,37.550727,126.922809
1,1,33617,2022-03-07,마포구,서울특별시 마포구 서교동 350-1,보도 중앙,2022-03-07,37.554865,126.925997
2,2,33619,2022-03-07,마포구,서울특별시 마포구 서교동 350-1,보도 중앙,2022-03-07,37.554865,126.925997
3,3,33620,2022-03-07,마포구,서울특별시 마포구 서교동 406,보도 중앙,2022-03-07,37.550727,126.922809
4,4,33625,2022-03-07,마포구,서울특별시 마포구 연남동 239-25,보도와 차도가 구분된 도로의 차도,2022-03-07,37.564473,126.923631


#### 주차구역 반경 1km? 이내 견인 신고 건수 카운트
- 참조 사이트 : https://wooiljeong.github.io/python/count_by_wgs84/

In [17]:
from geopy.distance import great_circle
import folium

class CountByWGS84:
    def __init__(self, df, lat, lon, dist=1) -> None:
        self.df = df
        self.lat = lat
        self.lon = lon
        self.dist = dist
    
    def filter_by_rectangle(self):
        lat_min = self.lat - 0.01 * self.dist
        lat_max = self.lat + 0.01 * self.dist
        lon_min = self.lon - 0.015 * self.dist
        lon_max = self.lon + 0.015 * self.dist
        self.points = [[lat_min, lon_min], 
                       [lat_max, lon_max]]
        result = self.df.loc[(self.df['lat'] > lat_min) & 
                             (self.df['lat'] < lat_max) & 
                             (self.df['lon'] > lon_min) & 
                             (self.df['lon'] < lon_max)]
        result.index = range(len(result))
        
        return result
    
    def filter_by_radius(self):
        tmp = self.filter_by_rectangle()
        center = (self.lat, self.lon)
        result = pd.DataFrame()
        for index, row in tmp.iterrows():
            point = (row['lat'], row['lon'])
            d = great_circle(center, point).kilometers
            if d <= self.dist:
                result = pd.concat([result, tmp.iloc[index, :].to_frame().T])
        result.index = range(len(result))

        return result
    
    def plot_by_rectangle(self, df):
        map = folium.Map(location=[self.lat, self.lon], zoom_start=14)
        for idx, row in df.iterrows():
            lat_ = row['lat']
            lon_ = row['lon']
            folium.Marker(location=[lat_, lon_], 
                          radius = 15, 
                          tooltip = row['유형']).add_to(map)
        folium.Rectangle(bounds=self.points,
                         color='#ff7800',
                         fill=True,
                         fill_color='#ffff00',
                         fill_opacity=0.2).add_to(map)
        return map
    
    def plot_by_radius(self, df):
        map = folium.Map(location=[self.lat, self.lon], zoom_start=14)
        for idx, row in df.iterrows():
            lat_ = row['lat']
            lon_ = row['lon']
            folium.Marker(location=[lat_, lon_], 
                          radius = 15, 
                          tooltip = row['유형']).add_to(map)
        folium.Circle(radius= dist * 1000,
                      location=[lat, lon], 
                      color='#ff7800', 
                      fill_color='#ffff00', 
                      fill_opacity=0.2).add_to(map)
        return map

In [18]:
lat = mapo_parking_df.loc[1, 'lat']
lon = mapo_parking_df.loc[1, 'lon']
dist = 1
cbw = CountByWGS84(mapo_df220307, lat, lon, dist)

In [19]:
result_rectangle = cbw.filter_by_rectangle()
result_radius = cbw.filter_by_radius()

In [20]:
print(f"""
{"="*50}
중심 위도: {cbw.lat}
중심 경도: {cbw.lon}
기준 거리: {cbw.dist} km
사각 범위 내 데이터 필터링 결과: {len(result_rectangle):,} 건
반경 범위 내 데이터 필터링 결과: {len(result_radius):,} 건
{"="*50}
""")


중심 위도: 37.5641966882799
중심 경도: 126.903275616221
기준 거리: 1 km
사각 범위 내 데이터 필터링 결과: 6 건
반경 범위 내 데이터 필터링 결과: 3 건



In [21]:
plot_1 = cbw.plot_by_rectangle(result_rectangle)
plot_1

In [22]:
plot_2 = cbw.plot_by_radius(result_radius)
plot_2